# Chapter 5: How to get the data

In [ ]:
import pandas as pd

In [ ]:
from urllib import request
fires_url = 'https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/RDS-2013-0009.4_SQLITE.zip'
request.urlretrieve(fires_url, filename='RDS-2013-0009.4_SQLITE.zip')

## How to import a CSV file from a website

In [ ]:
mortality_url = "https://data.cdc.gov/api/views/v6ab-adf5/rows.csv?accessType=DOWNLOAD"
mortality_data = pd.read_csv(mortality_url)

## How to import the first sheet of an Excel file

In [ ]:
# NOTE: This code example is presented later in this Notebook, after the Excel file has been downloaded

## How to download a file to disk before importing it

In [ ]:
from urllib import request
polls_url = \
    'http://projects.fivethirtyeight.com/general-model/president_general_polls_2016.csv'
request.urlretrieve(polls_url, filename='president_polls_2016.csv')

In [ ]:
polls = pd.read_csv('president_polls_2016.csv')
polls.head(2)

## How to work with a zip file on disk

### Download the zip file to disk

In [ ]:
# NOTE: This code may take a long time to run
from urllib import request
zip_url = 'https://www.bls.gov/oes/special.requests/oesm18all.zip'
request.urlretrieve(zip_url, filename='oesm18all.zip')

### Extract the files and list their names

In [ ]:
from zipfile import ZipFile
file_names = list() 
with ZipFile('oesm18all.zip', mode='r') as zip:
    zip.extractall()
    for file in zip.infolist():
        file_names.append(file.filename)
        print(file.filename, file.compress_size, file.file_size)

### Two ways to read an extracted file into a DataFrame

In [ ]:
# NOTE: This code may take a long time to run
jobs = pd.read_excel("oesm18all/all_data_M_2018.xlsx")

In [29]:
# NOTE: This code may take a long time to run
jobs.info(verbose=True, memory_usage='deep', show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403895 entries, 0 to 403894
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   area          403895 non-null  int64 
 1   area_title    403895 non-null  object
 2   area_type     403895 non-null  int64 
 3   naics         403895 non-null  object
 4   naics_title   403895 non-null  object
 5   i_group       403895 non-null  object
 6   own_code      403895 non-null  int64 
 7   occ_code      403895 non-null  object
 8   occ_title     403895 non-null  object
 9   o_group       403895 non-null  object
 10  tot_emp       403895 non-null  object
 11  emp_prse      403895 non-null  object
 12  jobs_1000     227944 non-null  object
 13  loc_quotient  227944 non-null  object
 14  pct_total     169080 non-null  object
 15  h_mean        403895 non-null  object
 16  a_mean        403895 non-null  object
 17  mean_prse     403895 non-null  object
 18  h_pct10       403895 non

In [30]:
jobs.columns = jobs.columns \
    .str.replace('^a_','annual_') \
    .str.replace('^h_','hourly_') \
    .str.replace('_pct','_percent')

C:\Users\andre\AppData\Local\Temp/ipykernel_31160/977832008.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs.columns = jobs.columns \
C:\Users\andre\AppData\Local\Temp/ipykernel_31160/977832008.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  jobs.columns = jobs.columns \


In [31]:
jobs.tot_emp.tail(3)


403892    170
403893    130
403894     **
Name: tot_emp, dtype: object

In [33]:
pd.to_numeric(jobs.tot_emp, errors='coerce').tail(3)


403892    170.0
403893    130.0
403894      NaN
Name: tot_emp, dtype: float64

In [34]:
import numpy as np
jobs.tot_emp.replace(to_replace=['*','**'], value=[np.nan,np.nan]) # ,inplace=True)
jobs.tot_emp.replace({'*':np.nan, '**':np.nan}, inplace=True)
jobs.hourly_median = jobs.hourly_median.replace(to_replace='#', value = 100)
jobs.hourly.replace(to_replace=np.nan, value=False, inplace=True)

## How to run queries against a database

In [ ]:
import sqlite3
fires_con = sqlite3.connect('Data/FPA_FOD_20170508.sqlite')
fires_cur = fires_con.cursor()

In [ ]:
fires_cur.execute(
    'SELECT name FROM sqlite_master WHERE type="table"').fetchall()

In [ ]:
fires_cur.execute(
    'SELECT * FROM Fires LIMIT 100').fetchall()

## How to use a SQL query to import data into a DataFrame

### Get information about a table

In [ ]:
fires_cur.execute('PRAGMA table_info(Fires)').fetchall()

### Import the data from a query into a DataFrame

In [ ]:
# NOTE: This code may take a long time to run
fires = pd.read_sql_query(
    '''SELECT STATE, FIRE_YEAR, DATETIME(DISCOVERY_DATE) AS DISCOVERY_DATE,
       FIRE_NAME, FIRE_SIZE, LATITUDE, LONGITUDE
    FROM Fires''', fires_con)
fires.head()

In [ ]:
len(fires)

## How to build DataFrames for the metadata and the data in a Stata file

### Get metadata from a Stata file

In [ ]:
!pip install pyreadstat

In [ ]:
import pyreadstat
gss_stata_filename = 'GSS7218_R3.DTA'
gss_empty, gss_meta = pyreadstat.read_dta(
    gss_stata_filename,
    metadataonly=True)
type(gss_meta)

### What the attributes of the metadata container can tell you

In [ ]:
print("Number of columns:", gss_meta.number_columns)
print("Number of rows:", gss_meta.number_rows)
print("Column names:", gss_meta.column_names)

### How to build a DataFrame for the column descriptions in the metadata

In [ ]:
meta_cols=pd.DataFrame(
    data=gss_meta.column_labels,
    index=gss_meta.column_names,
    columns=['description'])
meta_cols.head(5)

### How to import columns of the data into a DataFrame

In [ ]:
gss_data = pd.read_stata('GSS7218_R3.DTA',
    columns=['year','id','wrkstat','hrs1','hrs2','evwork','wrkslf','wrkgovt'])
gss_data.tail()

## How to download a JSON file

In [24]:
import json
from urllib import request
shots_url = 'https://www.murach.com/python_analysis/shots.json'
request.urlretrieve(shots_url, filename='shots.json')

('shots.json', <http.client.HTTPMessage at 0x1ca9ff062b0>)

## How to build a DataFrame for the data

In [25]:
with open('shots.json') as json_data:
    shots = json.load(json_data)

In [28]:
allRows = shots['resultSets'][0]['rowSet']
columnHeaders = [x.lower() for x in shots['resultSets'][0]['headers']]
shots = pd.DataFrame(data=allRows, columns=columnHeaders)
shots.head()

,grid_type,game_id,game_event_id,player_id,player_name,team_id,team_name,period,minutes_remaining,seconds_remaining,...,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_attempted_flag,shot_made_flag,game_date,htm,vtm
0,Shot Chart Detail,0020900015,4,201939,Stephen Curry,1610612744,Golden State Warriors,1,11,25,...,Right Side Center(RC),24+ ft.,26,99,249,1,0,20091028,GSW,HOU
1,Shot Chart Detail,0020900015,17,201939,Stephen Curry,1610612744,Golden State Warriors,1,9,31,...,Left Side Center(LC),16-24 ft.,18,-122,145,1,1,20091028,GSW,HOU
2,Shot Chart Detail,0020900015,53,201939,Stephen Curry,1610612744,Golden State Warriors,1,6,2,...,Center(C),8-16 ft.,14,-60,129,1,0,20091028,GSW,HOU
3,Shot Chart Detail,0020900015,141,201939,Stephen Curry,1610612744,Golden State Warriors,2,9,49,...,Left Side(L),16-24 ft.,19,-172,82,1,0,20091028,GSW,HOU
4,Shot Chart Detail,0020900015,249,201939,Stephen Curry,1610612744,Golden State Warriors,2,2,19,...,Left Side Center(LC),16-24 ft.,16,-68,148,1,0,20091028,GSW,HOU


In [27]:
shots['resultSets'][1]

{'name': 'LeagueAverages',
 'headers': ['GRID_TYPE',
  'SHOT_ZONE_BASIC',
  'SHOT_ZONE_AREA',
  'SHOT_ZONE_RANGE',
  'FGA',
  'FGM',
  'FG_PCT'],
 'rowSet': [['League Averages',
   'Above the Break 3',
   'Back Court(BC)',
   'Back Court Shot',
   2052,
   71,
   0.035],
  ['League Averages',
   'Above the Break 3',
   'Center(C)',
   '24+ ft.',
   170403,
   58640,
   0.344],
  ['League Averages',
   'Above the Break 3',
   'Left Side Center(LC)',
   '24+ ft.',
   286101,
   99899,
   0.349],
  ['League Averages',
   'Above the Break 3',
   'Right Side Center(RC)',
   '24+ ft.',
   281513,
   99450,
   0.353],
  ['League Averages',
   'Backcourt',
   'Back Court(BC)',
   'Back Court Shot',
   8621,
   231,
   0.027],
  ['League Averages',
   'In The Paint (Non-RA)',
   'Center(C)',
   '8-16 ft.',
   142361,
   58460,
   0.411],
  ['League Averages',
   'In The Paint (Non-RA)',
   'Center(C)',
   'Less Than 8 ft.',
   405784,
   158154,
   0.39],
  ['League Averages',
   'In The Paint 